In [1]:
import pandas as pd
import spacy
from spacy.training.example import Example

In [2]:
df = pd.read_excel("filtered.xlsx")

In [3]:
df.head()

,Unnamed: 0,product_title,product_overview_dict,Graphics Coprocessor,Brand,processed_title,processed_gc,start_end_positions,c_entities
0,0,GIGABYTE GeForce RTX 4070 Super Gaming OC 12G ...,"{'Graphics Coprocessor': 'RTX 4070 SUPER', 'Br...",RTX 4070 SUPER,GIGABYTE,gigabyte geforce rtx 4070 super gaming oc 12g ...,rtx 4070 super,"[17, 31, 'Coprocessor']","{'entities': [[17, 31, 'Coprocessor']]}"
1,1,NVIDIA Quadro 600 by PNY 1GB DDR3 PCI Express ...,"{'Graphics Coprocessor': 'NVIDIA Quadro', 'Bra...",NVIDIA Quadro,PNY,nvidia quadro 600 by pny 1gb ddr3 pci express ...,nvidia quadro,"[0, 13, 'Coprocessor']","{'entities': [[0, 13, 'Coprocessor']]}"
2,2,ASUS KO NVIDIA GeForce RTX 3070 OC Edition 8GB...,{'Graphics Coprocessor': 'NVIDIA GeForce RTX 3...,NVIDIA GeForce RTX 3070,ASUS,asus ko nvidia geforce rtx 3070 oc edition 8gb...,nvidia geforce rtx 3070,"[8, 31, 'Coprocessor']","{'entities': [[8, 31, 'Coprocessor']]}"
3,3,NVIDIA Quadro 2000D by PNY 1GB GDDR5 PCI Expre...,"{'Graphics Coprocessor': 'NVIDIA Quadro', 'Bra...",NVIDIA Quadro,PNY,nvidia quadro 2000d by pny 1gb gddr5 pci expre...,nvidia quadro,"[0, 13, 'Coprocessor']","{'entities': [[0, 13, 'Coprocessor']]}"
4,4,"EVGA GeForce RTX 2060 SC, Overclocked, 2.75 Sl...",{'Graphics Coprocessor': 'NVIDIA GeForce RTX 2...,NVIDIA GeForce RTX 2060,EVGA,evga geforce rtx 2060 sc overclocked 275 slot ...,nvidia geforce rtx 2060,"[3, 21, 'Coprocessor']","{'entities': [[3, 21, 'Coprocessor']]}"


In [4]:
df['c_entities'][3]

"{'entities': [[0, 13, 'Coprocessor']]}"

In [4]:
df.head()

,product_title,product_overview,Graphics Coprocessor,Brand,start_end_positions,brand_start_end_positions,combined_entities
0,"EVGA GeForce RTX 2060 SC, Overclocked, 2.75 Sl...","[['Graphics Coprocessor', 'NVIDIA GeForce RTX ...",NVIDIA GeForce RTX 2060,EVGA,"[3, 20, 'Coprocessor']","[0, 3, 'Brand']","{'entities': [[3, 20, 'Coprocessor'], [0, 3, '..."
1,PNY GeForce RTX™ 3050 8GB Verto Dual Fan Graph...,"[['Graphics Coprocessor', 'Nvidia RTX3050'], [...",Nvidia RTX3050,PNY,"[11, 14, 'Coprocessor']","[0, 2, 'Brand']","{'entities': [[11, 14, 'Coprocessor'], [0, 2, ..."
2,"SHOWKINGS Radeon RX 580 8GB Graphics Card, 256...","[['Graphics Coprocessor', 'AMD Radeon RX 580']...",AMD Radeon RX 580,SHOWKINGS,"[9, 22, 'Coprocessor']","[0, 8, 'Brand']","{'entities': [[9, 22, 'Coprocessor'], [0, 8, '..."
3,MSI Gaming GeForce GTX 1630 4GB GDRR6 64-Bit H...,"[['Graphics Coprocessor', 'NVIDIA GeForce GTX ...",NVIDIA GeForce GTX 1630,MSI,"[10, 26, 'Coprocessor']","[0, 2, 'Brand']","{'entities': [[10, 26, 'Coprocessor'], [0, 2, ..."
4,"SHOWKINGS GeForce GT 730 4GB Graphics Card, 12...","[['Graphics Coprocessor', 'NVIDIA GeForce GT 7...",NVIDIA GeForce GT 730,SHOWKINGS,"[9, 23, 'Coprocessor']","[0, 8, 'Brand']","{'entities': [[9, 23, 'Coprocessor'], [0, 8, '..."


In [7]:
len(df['Brand'].unique())

38

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   product_title              1376 non-null   object
 1   product_overview           1376 non-null   object
 2   Graphics Coprocessor       1376 non-null   object
 3   Brand                      1376 non-null   object
 4   start_end_positions        1376 non-null   object
 5   brand_start_end_positions  1376 non-null   object
 6   combined_entities          1376 non-null   object
dtypes: object(7)
memory usage: 76.7+ KB


In [21]:
df.dropna(inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, 0 to 788
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0.1               787 non-null    float64
 1   Unnamed: 0                 787 non-null    float64
 2   product_title              787 non-null    object 
 3   Graphics Coprocessor       787 non-null    object 
 4   Brand                      787 non-null    object 
 5   start_end_positions        787 non-null    object 
 6   brand_start_end_positions  787 non-null    object 
 7   c_entities                 787 non-null    object 
dtypes: float64(2), object(6)
memory usage: 55.3+ KB


In [5]:
import ast
import warnings
from spacy.scorer import Scorer
from sklearn.model_selection import train_test_split

# Ignore all warnings
warnings.filterwarnings("ignore")

nlp = spacy.load("en_core_web_sm")
#ner = nlp.add_pipe("ner")
ner = nlp.get_pipe("ner")
ner.add_label("Coprocessor")
#ner.add_label("Brand")

train_data=[]
for i, row in df.iterrows():
    text = row["product_title"]
    
    # Convert string representation of dictionary to actual dictionary
    annotations = ast.literal_eval(row["c_entities"])
    
    entities = annotations.get("entities", [])  # Use get to handle missing 'entities' key
    
    train_data.append(Example.from_dict(nlp.make_doc(text), {"entities": entities}))




In [6]:
import random
n_iter = 10

for _ in range(n_iter):
    # Shuffle training data for each iteration
    spacy.util.fix_random_seed(1)
    random.shuffle(train_data)

    # Create Example objects and update the model
    for example in train_data:
        nlp.update([example], drop=0.7)  # Drop rate can be adjusted
nlp.to_disk("coprocessor_ner")

In [14]:
from spacy import displacy
nlp = spacy.load("coprocessor_ner")
text = 'HP Victus Gaming Laptop, 15.6 FHD 144Hz IPS Display, Intel Core i5-13420H, 32GB RAM, 1TB PCIe SSD, NVIDIA GeForce RTX 3050, Backlit Keyboard, Wi-Fi 6, Win 11 Pro, Blue, 32GB Hotface USB Card'
doc = nlp(text)
displacy.render(doc, style="ent")

In [24]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [12]:
df.head(25)

,Unnamed: 0,product_title,product_overview_dict,Graphics Coprocessor,Brand,processed_title,processed_gc,start_end_positions,c_entities
0,0,GIGABYTE GeForce RTX 4070 Super Gaming OC 12G ...,"{'Graphics Coprocessor': 'RTX 4070 SUPER', 'Br...",RTX 4070 SUPER,GIGABYTE,gigabyte geforce rtx 4070 super gaming oc 12g ...,rtx 4070 super,"[17, 31, 'Coprocessor']","{'entities': [[17, 31, 'Coprocessor']]}"
1,1,NVIDIA Quadro 600 by PNY 1GB DDR3 PCI Express ...,"{'Graphics Coprocessor': 'NVIDIA Quadro', 'Bra...",NVIDIA Quadro,PNY,nvidia quadro 600 by pny 1gb ddr3 pci express ...,nvidia quadro,"[0, 13, 'Coprocessor']","{'entities': [[0, 13, 'Coprocessor']]}"
2,2,ASUS KO NVIDIA GeForce RTX 3070 OC Edition 8GB...,{'Graphics Coprocessor': 'NVIDIA GeForce RTX 3...,NVIDIA GeForce RTX 3070,ASUS,asus ko nvidia geforce rtx 3070 oc edition 8gb...,nvidia geforce rtx 3070,"[8, 31, 'Coprocessor']","{'entities': [[8, 31, 'Coprocessor']]}"
3,3,NVIDIA Quadro 2000D by PNY 1GB GDDR5 PCI Expre...,"{'Graphics Coprocessor': 'NVIDIA Quadro', 'Bra...",NVIDIA Quadro,PNY,nvidia quadro 2000d by pny 1gb gddr5 pci expre...,nvidia quadro,"[0, 13, 'Coprocessor']","{'entities': [[0, 13, 'Coprocessor']]}"
4,4,"EVGA GeForce RTX 2060 SC, Overclocked, 2.75 Sl...",{'Graphics Coprocessor': 'NVIDIA GeForce RTX 2...,NVIDIA GeForce RTX 2060,EVGA,evga geforce rtx 2060 sc overclocked 275 slot ...,nvidia geforce rtx 2060,"[3, 21, 'Coprocessor']","{'entities': [[3, 21, 'Coprocessor']]}"
5,5,ZOTAC Gaming GeForce RTX™ 3060 Ti Twin Edge OC...,{'Graphics Coprocessor': 'NVIDIA GeForce RTX 3...,NVIDIA GeForce RTX 3060 Ti,ZOTAC,zotac gaming geforce rtxtm 3060 ti twin edge o...,nvidia geforce rtx 3060 ti,"[12, 24, 'Coprocessor']","{'entities': [[12, 24, 'Coprocessor']]}"
6,6,MSI Gaming AMD Radeon RX 6600 128-bit 8GB GDDR...,"{'Graphics Coprocessor': 'AMD Radeon RX 6600',...",AMD Radeon RX 6600,MSI,msi gaming amd radeon rx 6600 128bit 8gb gddr6...,amd radeon rx 6600,"[11, 29, 'Coprocessor']","{'entities': [[11, 29, 'Coprocessor']]}"
7,7,"GALAX GeForce RTX™ 4070 1-Click OC 2X, Xtreme ...",{'Graphics Coprocessor': 'NVIDIA GeForce RTX 4...,NVIDIA GeForce RTX 4070,GALAX,galax geforce rtxtm 4070 1click oc 2x xtreme t...,nvidia geforce rtx 4070,"[5, 17, 'Coprocessor']","{'entities': [[5, 17, 'Coprocessor']]}"
8,8,ASUS TUF Gaming GeForce RTX 3050 OC Edition Ga...,{'Graphics Coprocessor': 'NVIDIA GeForce RTX 3...,NVIDIA GeForce RTX 3050,ASUS,asus tuf gaming geforce rtx 3050 oc edition ga...,nvidia geforce rtx 3050,"[15, 32, 'Coprocessor']","{'entities': [[15, 32, 'Coprocessor']]}"
9,9,Nvidia Quadro K2200 4GB Check 128-bit PCI Expr...,{'Graphics Coprocessor': 'NVIDIA Quadro K2200'...,NVIDIA Quadro K2200,NVIDIA,nvidia quadro k2200 4gb check 128bit pci expre...,nvidia quadro k2200,"[0, 19, 'Coprocessor']","{'entities': [[0, 19, 'Coprocessor']]}"
